In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
import re
from datetime import datetime, timedelta
import time
import json
from collections import OrderedDict

In [ ]:
file_data = OrderedDict()
options = webdriver.ChromeOptions()
path = "C:\\Users\\DeepLearning_4\\Desktop\\driver\\chromedriver.exe"

driver = webdriver.Chrome(path , options=options)
driver.set_window_size(360, 740)

In [ ]:
def make_link():
    page = 1
    links = []
    while True :
        
        if page == 67:
            break
        driver.get('https://kumoh42.com/index.php?mid=park&page=' + str(page))
        req = driver.page_source
        soup = BeautifulSoup(req, 'html.parser')

        for tr in soup.select("tbody > tr") :
            if not tr.find(class_ = re.compile("notice")) : 
                link = tr.find(href=re.compile("document_srl"))
                links.append(link.attrs['href'])

        for link in links:
            print(link)
            
        page += 1
        
    return links

In [ ]:
links = make_link()

In [ ]:
output = []
for link in links :
    driver.get(link)
    
    req = driver.page_source
    soup = BeautifulSoup(req, 'html.parser')
    
    commentNum = soup.select_one('#comment > div.eq.flex.flex-wrap > div.ed.flex.flex-left.flex-wrap.margin-top-small.margin-bottom-small > h4').text
    num = re.sub("개의 댓글", "", commentNum)
    if int(num)==0 :
        continue
    
    title = soup.select_one("#main > div:nth-child(2) > div > div > div.col-md-9 > div > div.ed.article-wrapper.inner-container > div.ed > div.ed.article-head.margin-bottom-large > h2 > a").text
    
    content = ""
    for data in soup.find_all(class_ = re.compile("^document")) : 
        content += data.text

    comments= []
    for data in soup.select("div.ed.comment-content > div > div:nth-child(2) > div > p"): 
        comments.append({
          "comment" : data.text   
        })
#     print(comments)
    
#     date = soup.select_one("#main > div:nth-child(2) > div > div > div.col-md-9 > div > div.ed.article-wrapper.inner-container > div.ed > div.ed.article-head.margin-bottom-large > div > div.ed.flex.flex-wrap > span.ed.text-muted.margin-right-small").text       
#     # ~시간 전 | ~일 전 이라고 표시되는 게시물 존재함  
#     if '전' in date:
#         now = datetime.now()
#         if '시간' in date:  # '~시간 전'
#             date = str(now.year) + '.' + str(now.month) + '.' + str(now.day)
#         else : # '~일 전'
#             date = re.sub("일 전", "", date)
    json_dic = dict()
    json_dic['title'] = title
    json_dic['content'] = content
    json_obj = dict()
    json_obj = comments
    json_dic['comments'] = json_obj
    
    output.append(json_dic)
#     print(json_dic)

#     data.append({
# #         "date" : date,
#         "title" : title,
#         "content" : content,
#         "comments" : comments
# #         "link" : link
#     }) 

In [ ]:
with open("금오사이.json", "w", encoding='UTF-8') as json_file:
    json.dump(output, json_file, indent=4, ensure_ascii = False)